In [120]:
import pandas as pd
import numpy as np
from datetime import datetime

In [121]:
data_path='D:\PythonDev\MyQuantFinProject\Data\AllThai-Oct22-Jan23.csv'


trade_plan_path='MasterConfigData\ListTradePlan.xlsx'
trade_sheet='July22-Now'

asset_comm_path='MasterConfigData\AssetInfo.xlsx'

result_path=f'ReportData\ReportTestPort_{trade_sheet}-SMALL.xlsx'


In [122]:
# df=pd.read_csv('ASP-SME_KKP-SM-TRade.csv',index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
# df.index.set_names('date',inplace=True)
df.columns=['symbol','date','price']

list_symbols=df['symbol'].unique()

print(list_symbols)
df.info()
print(df.tail())


['ASP-T12' 'ASP-SME' 'ASP-THEQ' 'K-MIDSMALL' 'KFTHAISM' 'KKP-ACT-EQ-A'
 'KFDYNAMIC' 'K-SET50' 'KKP-SM-CAP' 'KKP-TQG' 'TISCOFLEXP' 'TLMSEQ-A'
 'TLEQ' 'TISCOMS-A' 'TSF-A']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6757 entries, 0 to 6756
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  6757 non-null   object        
 1   date    6757 non-null   datetime64[ns]
 2   price   6757 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 158.5+ KB
     symbol       date  price
6752  TSF-A 2023-01-23  85.10
6753  TSF-A 2023-01-24  85.58
6754  TSF-A 2023-01-25  85.08
6755  TSF-A 2023-01-26  84.25
6756  TSF-A 2023-01-27  85.30


In [123]:
# Fix Symbol as you want if not compare all in excel
list_symbols=['KKP-SM-CAP','KKP-ACT-EQ-A','KKP-TQG','TISCOFLEXP','TISCOMS-A','TSF-A','ASP-T12','ASP-SME','ASP-THEQ']
# list_symbols=['KKP-SM-CAP']

dfAssetInfo=pd.read_excel(asset_comm_path,engine="openpyxl",sheet_name='AssetInfo',\
                    index_col='symbol', dtype ={'buy_comm_pct': np.float64,'sell_comm_pct': np.float64})

dfNoComissionInfo= [ item for item in list_symbols if item not in list(dfAssetInfo.index)]
if  len(dfNoComissionInfo)>0:
  raise Exception(f"{dfNoComissionInfo} didn't found commision info in {asset_comm_path}")
else:
  dfAssetInfo=dfAssetInfo.loc[list(list_symbols),:]
  print("Commmision info")
  print(dfAssetInfo)  

Commmision info
              buy_comm_pct  sell_comm_pct
symbol                                   
KKP-SM-CAP            1.00            0.0
KKP-ACT-EQ-A          0.50            0.0
KKP-TQG               0.50            0.0
TISCOFLEXP            1.00            0.0
TISCOMS-A             1.00            0.0
TSF-A                 1.05            0.0
ASP-T12               1.25            0.0
ASP-SME               1.25            0.0
ASP-THEQ              1.00            0.0


In [124]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=trade_sheet, \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})



#we have to add last reord 100 % as of last dat in Amibroker database
buy_sum_act=dfPlan.query("action=='b'").portion_pct.sum()
sell_sum_act=dfPlan.query("action=='s'").portion_pct.sum()
all_portion=0
if buy_sum_act<sell_sum_act:
  raise Exception("Not allow Buy <= Sell")   
else:
    if buy_sum_act>sell_sum_act:
       print("=============No explicite lasst sell , we will append SellAll-At-LastPrice=============")
       sellAllAtNow=buy_sum_act-sell_sum_act

       dfPlan=pd.concat([dfPlan,pd.DataFrame(columns=list(dfPlan.columns), data=[ [df['date'].max(),sellAllAtNow,'s']])]) 
       dfPlan=dfPlan.reset_index(drop=True)

    tradeDateList= [ d for d in list(dfPlan['trade_date']) ]    
    all_portion=buy_sum_act   
    print(f"portion = {all_portion} on")
    
print("================================================================================================")
print(f"No Buy/Sell {dfPlan.groupby(['action']).size()}")
print(dfPlan)


=============No explicite lasst sell , we will append SellAll-At-LastPrice=============
portion = 100.0 on
No Buy/Sell action
b    9
s    1
dtype: int64
  trade_date  portion_pct action
0 2022-07-22         10.0      b
1 2022-08-08         10.0      b
2 2022-08-22         20.0      b
3 2022-09-27         10.0      b
4 2022-10-20         10.0      b
5 2022-11-01         10.0      b
6 2022-11-25         10.0      b
7 2022-12-30         10.0      b
8 2023-01-06         10.0      b
9 2023-01-27        100.0      s


In [125]:
TotalBudget=100000
rrr=TotalBudget % all_portion
if rrr !=0 :
  raise Exception(f"Your Budget {TotalBudget} is  not proportiona;  Portion with {all_portion}")
else:
  print(f"Your Budget {TotalBudget} is  proportional  Portion with {all_portion}")

Your Budget 100000 is  proportional  Portion with 100.0


In [126]:
def set_commision_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    comm_type='buy_comm_pct'
  elif  item['action']=='s':
    comm_type='sell_comm_pct'
  try:
     return dfAssetInfo.loc[x,comm_type]
  except Exception as ex:
     print(f"Error : no {x} to get commission")
     return 0
def get_qty_buy(item):
    xCost=(item['portion_pct']/all_portion)*TotalBudget
    xPrice=item['price_on_comm']
    qty=xCost/xPrice
    return qty
def get_qty_sell(item,TotalQty):
    qty=(item['portion_pct']/all_portion)*TotalQty
    return qty


In [127]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])
for symbol in list_symbols:
    
    
    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    noPlanedTrade= len(tradeDateList)
    noActualTrade=len(dfBS)
    if noPlanedTrade==len(dfBS):
    
        print(f"{symbol} Buy/Sell ={noPlanedTrade}")
        dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')

        dfBS['comm']=dfBS.apply(set_commision_by_action,axis=1)
        dfBS['price_on_comm']=dfBS.apply(lambda item : item['price']*(1+ (item['comm']/100)) if item['action']=='b' else item['price']*(1-(item['comm']/100)), axis=1)
        dfBS['price_on_comm']=dfBS['price_on_comm'].round(2)

        dfBS=dfBS.drop(columns=['trade_date'])
        #print(dfBS)
    
        print("================Buy=================")
        buyDF=dfBS.query("action=='b'")    
        # cal qury and actual value and  total_val and totl qury
        buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
        buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
        buyDF['action_value']=buyDF['value']
        buyDF['action_qty']=buyDF['qty']
        print(buyDF[['date' ,'price_on_comm','qty','value','portion_pct']])

        TotalQty=buyDF['qty'].sum()
        TotalValue=round(buyDF['value'].sum(),2)
        AvgPrice=round(TotalValue/TotalQty,4)
        # print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")

        print("=================Sell=================")
        sellDF=dfBS.query("action=='s'")  
        sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
        sellDF['value']=sellDF['qty']*sellDF['price_on_comm']
        sellDF['action_value']=sellDF['value']*-1
        sellDF['action_qty']=sellDF['qty']*-1
        print(sellDF[['date' ,'price_on_comm','qty','value','portion_pct']])

        SellTotalQty=sellDF['qty'].sum()
        SellTotalValue=round(sellDF['value'].sum(),2)
        SellAvgPrice=round(SellTotalValue/SellTotalQty,4)

        # print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")

        PL=round(SellTotalValue-TotalValue,0)
        PCT_PL=round(PL/TotalValue*100,2)

        summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,'AvgSellPrice':SellAvgPrice, \
                      'PL':PL,'Percent_PL':PCT_PL
                     }
        dfSummary=dfSummary.append(summary_data,ignore_index=True)

        dfTran=pd.concat([buyDF,sellDF],axis=0)
        dfTran=dfTran.sort_values(by=['date'])
        dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)

        dict_dfTran[symbol]=dfTran
    else:
        print(f"{symbol} has been exclued from trading process due to{noActualTrade}<> {noPlanedTrade}")


KKP-SM-CAP Buy/Sell =10
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2022-07-22          16.81   594.883998  10000.0         10.0
1 2022-08-08          17.40   574.712644  10000.0         10.0
2 2022-08-22          17.65  1133.144476  20000.0         20.0
3 2022-09-27          17.69   565.291125  10000.0         10.0
4 2022-10-20          17.17   582.411182  10000.0         10.0
5 2022-11-01          17.35   576.368876  10000.0         10.0
6 2022-11-25          17.58   568.828214  10000.0         10.0
7 2022-12-30          17.23   580.383053  10000.0         10.0
8 2023-01-06          17.25   579.710145  10000.0         10.0
=================Sell=================
        date  price_on_comm          qty         value  portion_pct
9 2023-01-27          17.21  5755.733712  99056.177186        100.0
KKP-ACT-EQ-A Buy/Sell =10
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2022-0

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

ASP-THEQ Buy/Sell =10
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2022-07-22          14.35   696.864111  10000.0         10.0
1 2022-08-08          14.86   672.947510  10000.0         10.0
2 2022-08-22          15.02  1331.557923  20000.0         20.0
3 2022-09-27          14.96   668.449198  10000.0         10.0
4 2022-10-20          14.85   673.400673  10000.0         10.0
5 2022-11-01          15.17   659.195781  10000.0         10.0
6 2022-11-25          14.53   688.231246  10000.0         10.0
7 2022-12-30          15.31   653.167864  10000.0         10.0
8 2023-01-06          15.46   646.830530  10000.0         10.0
=================Sell=================
        date  price_on_comm          qty          value  portion_pct
9 2023-01-27          15.24  6690.644837  101965.427316        100.0


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [128]:
for symbol,tranDF in dict_dfTran.items():
  tranDF['net_value']=tranDF['action_value'].cumsum(axis = 0) 
  tranDF['net_qty']=tranDF['action_qty'].cumsum(axis = 0)  
  tranDF['avg_price']=round( tranDF['net_value']/tranDF['net_qty'],4)
  tranDF=tranDF.sort_values(by=['date'])
  print(tranDF[['date' ,'action','price_on_comm','qty','net_qty','value','net_value','avg_price']])

        date action  price_on_comm          qty      net_qty    value  \
0 2022-07-22      b          16.81   594.883998   594.883998  10000.0   
1 2022-08-08      b          17.40   574.712644  1169.596641  10000.0   
2 2022-08-22      b          17.65  1133.144476  2302.741117  20000.0   
3 2022-09-27      b          17.69   565.291125  2868.032242  10000.0   
4 2022-10-20      b          17.17   582.411182  3450.443424  10000.0   
5 2022-11-01      b          17.35   576.368876  4026.812301  10000.0   
6 2022-11-25      b          17.58   568.828214  4595.640514  10000.0   
7 2022-12-30      b          17.23   580.383053  5176.023567  10000.0   
8 2023-01-06      b          17.25   579.710145  5755.733712  10000.0   
9 2023-01-27      s          17.21  5755.733712     0.000000  99056.0   

   net_value  avg_price  
0    10000.0    16.8100  
1    20000.0    17.0999  
2    40000.0    17.3706  
3    50000.0    17.4336  
4    60000.0    17.3891  
5    70000.0    17.3835  
6    80000.0  

In [129]:
writer=pd.ExcelWriter(result_path,engine='xlsxwriter') 
dfSummary=dfSummary.sort_values(by=['Percent_PL'],ascending=False)
dfSummary.to_excel(writer, sheet_name="Summary",index=False)

for symbol,tranDF in dict_dfTran.items():
 tranDF.to_excel(writer, sheet_name=symbol,index=False) 
 #print(tranDF)
writer.save()

writer.close()

print("Successfully")


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\1780546000.py:8: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


Successfully


D:\ProgramData\Anaconda3\envs\myquant_exp\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [130]:
dfSummary

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL
5,TSF-A,100000.0,81.1937,105057.43,85.30,5057.0,5.06
7,ASP-SME,100000.0,17.4688,103441.32,18.07,3441.0,3.44
4,TISCOMS-A,100000.0,20.4437,103308.13,21.12,3308.0,3.31
1,KKP-ACT-EQ-A,100000.0,18.8634,102420.32,19.32,2420.0,2.42
3,TISCOFLEXP,100000.0,60.1233,102056.87,61.36,2057.0,2.06
8,ASP-THEQ,100000.0,14.9462,101965.43,15.24,1965.0,1.96
6,ASP-T12,100000.0,10.8313,101372.66,10.98,1373.0,1.37
2,KKP-TQG,100000.0,8.9605,99436.90,8.91,-563.0,-0.56
0,KKP-SM-CAP,100000.0,17.3740,99056.18,17.21,-944.0,-0.94
